# ❤️ Case Study: Heart Disease Classification

---

## A Life-Saving Problem

Heart disease is the #1 cause of death worldwide. Early detection saves lives!

> **The Question**: Given a patient's clinical data, can we predict if they have heart disease?

This is exactly what hospitals use machine learning for - helping doctors make better decisions!

---

## Why Logistic Regression?

This is a **binary classification** problem:
- Class 0: No heart disease
- Class 1: Heart disease present

Logistic Regression is perfect because:
1. Outputs **probabilities** (not just classes)
2. **Highly interpretable** (doctors can understand the coefficients)
3. Works well for **medical diagnosis** where we need to expother decisions

💡 **Key Insight**: In medicine, a black-box model that says "you're sick" isn't helpful. Doctors need to know WHY!

In [ ]:
# ═══════════════════════════════════════════════════════════════
# 📦 STEP 1: LOAD DATA AND LIBRARIES
# ═══════════════════════════════════════════════════════════════
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import (accuracy_score, classification_report, confusion_matrix, 
                             roc_curve, auc, precision_recall_curve, f1_score)
from sklearn.pipeline import Pipeline
import warnings
warnings.filterwarnings('ignore')

print("📥 Loading Heart Disease Dataset from OpenML...")
print("="*50)
heart = fetch_openml(name='heart-statlog', version=1, as_frame=True)
df = heart.frame

print(f"✅ Loaded {len(df)} patient records")
print(f"\n📋 Features available: {list(df.columns)}")
print(f"\n📊 First 5 patients:")
print(df.head())
print(f"\n💾 Dataset shape: {df.shape}")

## 📊 Step 2: Exploratory Data Analysis

Before building any model, let's understand our data. In medical ML, EDA is CRITICAL because:
- We need to catch data quality issues (missing values, outliers)
- We need to understand feature distributions
- We need to check for class imbalance (common in disease detection!)

> **Golden Rule**: "If you don't understand your data, your model won't either."

In [ ]:
# ═══════════════════════════════════════════════════════════════
# 📊 EXPLORATORY DATA ANALYSIS
# ═══════════════════════════════════════════════════════════════

# 1. Dataset Overview
print("📋 DATASET OVERVIEW")
print("="*50)
print(f"Shape: {df.shape}")
print(f"\nData Types:\n{df.dtypes}")
print(f"\nMissing Values:\n{df.isnull().sum()}")

# 2. Target Distribution
print("\n📊 TARGET DISTRIBUTION")
y = (df['target'] == '2').astype(int)
print(f"   No Heart Disease (0): {(y == 0).sum()} ({(y == 0).mean()*100:.1f}%)")
print(f"   Heart Disease (1): {(y == 1).sum()} ({(y == 1).mean()*100:.1f}%)")

# 3. Visualizations
fig, axes = plt.subplots(2, 3, figsize=(15, 10))

# Age distribution by class
ax = axes[0, 0]
for label in [0, 1]:
    subset = df[y == label]['age'].astype(float)
    ax.hist(subset, bins=20, alpha=0.7, label=f'Class {label}')
ax.set_xlabel('Age')
ax.set_ylabel('Frequency')
ax.set_title('Age Distribution by Heart Disease Status')
ax.legend()

# Correlation heatmap (numeric features only)
ax = axes[0, 1]
numeric_df = df.select_dtypes(include=[np.number])
if len(numeric_df.columns) > 0:
    sns.heatmap(numeric_df.corr(), annot=True, fmt='.2f', cmap='coolwarm', ax=ax)
    ax.set_title('Feature Correlations')

# Class balance pie chart
ax = axes[0, 2]
y.value_counts().plot.pie(ax=ax, autopct='%1.1f%%', colors=['lightblue', 'salmon'])
ax.set_title('Class Distribution')
ax.set_ylabel('')

# Chest pain type distribution
ax = axes[1, 0]
if 'chest' in df.columns:
    pd.crosstab(df['chest'], y).plot(kind='bar', ax=ax)
    ax.set_title('Chest Pain Type vs Heart Disease')
    ax.set_xlabel('Chest Pain Type')

# Resting BP boxplot
ax = axes[1, 1]
if 'resting_blood_pressure' in df.columns:
    df.boxplot(column='resting_blood_pressure', by=y, ax=ax)
    ax.set_title('Resting BP by Heart Disease Status')

# Max heart rate
ax = axes[1, 2]
if 'max_heart_rate' in df.columns:
    df.boxplot(column='max_heart_rate', by=y, ax=ax)
    ax.set_title('Max Heart Rate by Heart Disease Status')

plt.suptitle('Heart Disease EDA', y=1.02, fontsize=14)
plt.tight_layout()
plt.show()

print("\n💡 Insights from EDA:")
print("   • Dataset is relatively balanced (good!)")
print("   • Age and max heart rate show clear differences between classes")
print("   • Chest pain type appears to be a strong predictor")

## 📋 Understanding the Features

Each feature is a clinical measurement that doctors commonly use for heart disease assessment:

| Feature | Description | Clinical Relevance |
|---------|-------------|-------------------|
| **age** | Patient age in years | Risk increases with age |
| **sex** | Male (1) or Female (0) | Males have higher risk |
| **chest** | Chest pain type (1-4) | Key diagnostic symptom |
| **resting_blood_pressure** | Resting BP (mm Hg) | High BP = higher risk |
| **serum_cholesterol** | Cholesterol level (mg/dl) | High cholesterol = higher risk |
| **fasting_blood_sugar** | >120 mg/dl (1=true) | Indicates diabetes risk |
| **resting_electrocardiographic_results** | ECG results (0-2) | Detects heart abnormalities |
| **max_heart_rate** | Max HR during exercise | Low max HR = higher risk |
| **exercise_induced_angina** | Chest pain during exercise | Strong predictor |
| **oldpeak** | ST depression from exercise | ECG abnormality marker |
| **slope** | Slope of ST segment | Heart function indicator |
| **number_of_major_vessels** | Vessels colored by fluoroscopy | More = worse prognosis |
| **thal** | Thalassemia blood disorder | Normal/Fixed/Reversible defect |
| **target** | Heart disease presence | 1=No disease, 2=Disease |

💡 **Key Insight**: These features are derived from real clinical tests - blood work, ECG, and exercise stress tests!

In [ ]:
# ═══════════════════════════════════════════════════════════════
# 🎯 STEP 3: DATA PREPARATION AND MODEL TRAINING
# ═══════════════════════════════════════════════════════════════

# Prepare features and target
X = df.drop('target', axis=1)
y = (df['target'] == '2').astype(int)  # 2 = has disease

print("📊 CLASS DISTRIBUTION")
print("="*50)
print(f"   No Heart Disease (0): {(y == 0).sum()} ({(y == 0).mean()*100:.1f}%)")
print(f"   Heart Disease (1): {(y == 1).sum()} ({(y == 1).mean()*100:.1f}%)")

# Feature Scaling (CRITICAL for Logistic Regression!)
print("\n🔧 SCALING FEATURES")
print("-"*50)
print("Why scale? Logistic Regression uses gradient descent.")
print("Features on different scales → slow convergence, biased coefficients!")

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Train-Test Split (stratified to maintain class balance)
X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y, test_size=0.2, random_state=42, stratify=y
)

print(f"\n📊 Data Split:")
print(f"   Training: {len(X_train)} samples")
print(f"   Testing: {len(X_test)} samples")

# Train Logistic Regression
print("\n🤖 TRAINING MODEL")
print("-"*50)
model = LogisticRegression(max_iter=1000, random_state=42)
model.fit(X_train, y_train)

print("✅ Model successfully trained!")
print(f"   Coefficients shape: {model.coef_.shape}")
print(f"   Intercept: {model.intercept_[0]:.4f}")

## 🔍 Step 4: Feature Importance Analysis

One of the biggest advantages of Logistic Regression in medicine is **interpretability**.

We can look at the coefficients to understand:
- Which features increase heart disease risk?
- Which features are protective?
- How much does each factor matter?

> **Medical Insight**: A model that says "your risk is 85%" isn't actionable. But a model that says "your high cholesterol and low exercise tolerance are the main risk factors" helps doctors create a treatment plan!

In [ ]:
# ═══════════════════════════════════════════════════════════════
# 🔍 FEATURE IMPORTANCE
# ═══════════════════════════════════════════════════════════════

# Get feature names and coefficients
feature_names = X.columns.tolist()
coefficients = model.coef_[0]

# Create importance dataframe
importance_df = pd.DataFrame({
    'Feature': feature_names,
    'Coefficient': coefficients,
    'Abs_Coefficient': np.abs(coefficients),
    'Odds_Ratio': np.exp(coefficients)
})
importance_df = importance_df.sort_values('Abs_Coefficient', ascending=False)

print("📊 FEATURE IMPORTANCE (by |coefficient|)")
print("="*60)
print(importance_df.to_string(index=False))

# Visualize
plt.figure(figsize=(10, 8))
colors = ['red' if c > 0 else 'blue' for c in importance_df['Coefficient']]
plt.barh(importance_df['Feature'], importance_df['Coefficient'], color=colors)
plt.xlabel('Coefficient (Log-Odds)', fontsize=12)
plt.ylabel('Feature', fontsize=12)
plt.title('Feature Importance: Risk Factors for Heart Disease\nRed = Increases Risk | Blue = Decreases Risk', fontsize=14)
plt.axvline(x=0, color='black', linestyle='--', alpha=0.5)
plt.tight_layout()
plt.show()

# Interpretation
print("\n💡 INTERPRETATION:")
print("-"*50)
for _, row in importance_df.head(5).iterrows():
    direction = "INCREASES" if row['Coefficient'] > 0 else "DECREASES"
    print(f"• {row['Feature']}: {direction} heart disease risk")
    print(f"  Odds Ratio: {row['Odds_Ratio']:.3f}")
    print(f"  (1-unit increase in scaled feature → odds multiply by {row['Odds_Ratio']:.3f})")
    print()

In [ ]:
# ═══════════════════════════════════════════════════════════════
# 📊 MODEL EVALUATION - CONFUSION MATRIX
# ═══════════════════════════════════════════════════════════════

# Make predictions
y_pred = model.predict(X_test)
y_proba = model.predict_proba(X_test)[:, 1]

print("📊 CLASSIFICATION REPORT")
print("="*60)
print(classification_report(y_test, y_pred, target_names=['No Disease', 'Heart Disease']))

# Confusion Matrix with detailed annotations
cm = confusion_matrix(y_test, y_pred)

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Standard confusion matrix
ax = axes[0]
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
            xticklabels=['No Disease', 'Heart Disease'], 
            yticklabels=['No Disease', 'Heart Disease'], ax=ax)
ax.set_title('Confusion Matrix', fontsize=14)
ax.set_xlabel('Predicted', fontsize=12)
ax.set_ylabel('Actual', fontsize=12)

# Annotated confusion matrix with medical context
ax = axes[1]
tn, fp, fn, tp = cm.ravel()
annotated = np.array([
    [f'TN\n{tn}\n(Correctly healthy)', f'FP\n{fp}\n(False alarm)'],
    [f'FN\n{fn}\n(MISSED DISEASE!)', f'TP\n{tp}\n(Correctly detected)']
])
sns.heatmap(cm, annot=annotated, fmt='', cmap='RdYlGn', 
            xticklabels=['Predicted Healthy', 'Predicted Disease'], 
            yticklabels=['Actually Healthy', 'Actually Disease'], ax=ax)
ax.set_title('Medical Context: Confusion Matrix', fontsize=14)

plt.tight_layout()
plt.show()

# Medical implications
print("\n🏥 MEDICAL IMPLICATIONS:")
print("-"*50)
print(f"   True Positives (TP): {tp} - Correctly identified sick patients ✅")
print(f"   True Negatives (TN): {tn} - Correctly identified healthy patients ✅")
print(f"   False Positives (FP): {fp} - Healthy patients flagged for extra tests (acceptable)")
print(f"   False Negatives (FN): {fn} - MISSED DISEASE CASES (dangerous!) ⚠️")
print(f"\n   Sensitivity (Recall): {tp/(tp+fn):.1%} of sick patients detected")
print(f"   Specificity: {tn/(tn+fp):.1%} of healthy patients correctly identified")

In [ ]:
# ═══════════════════════════════════════════════════════════════
# 📈 ROC CURVE AND AUC
# ═══════════════════════════════════════════════════════════════

# Calculate ROC curve
fpr, tpr, thresholds = roc_curve(y_test, y_proba)
roc_auc = auc(fpr, tpr)

# Find optimal threshold (Youden's J statistic)
j_scores = tpr - fpr
optimal_idx = np.argmax(j_scores)
optimal_threshold = thresholds[optimal_idx]

plt.figure(figsize=(10, 8))

# Main ROC curve
plt.plot(fpr, tpr, color='darkorange', linewidth=2.5, 
         label=f'ROC Curve (AUC = {roc_auc:.3f})')
plt.plot([0, 1], [0, 1], 'k--', linewidth=1, label='Random Classifier')

# Mark optimal point
plt.scatter(fpr[optimal_idx], tpr[optimal_idx], color='red', s=150, zorder=5,
            label=f'Optimal Threshold = {optimal_threshold:.2f}')

# Add some threshold annotations
for i, thresh in enumerate([0.3, 0.5, 0.7]):
    idx = np.argmin(np.abs(thresholds - thresh))
    plt.annotate(f't={thresh}', (fpr[idx], tpr[idx]), 
                 textcoords="offset points", xytext=(10, -10), fontsize=9)

plt.xlabel('False Positive Rate (1 - Specificity)', fontsize=12)
plt.ylabel('True Positive Rate (Sensitivity)', fontsize=12)
plt.title('ROC Curve: Heart Disease Detection\nHigher AUC = Better Discrimination', fontsize=14)
plt.legend(loc='lower right', fontsize=11)
plt.grid(True, alpha=0.3)

# Add reference lines
plt.axhline(y=0.9, color='gray', linestyle=':', alpha=0.5)
plt.axvline(x=0.1, color='gray', linestyle=':', alpha=0.5)

plt.tight_layout()
plt.show()

# Performance summary
print("\n🎯 MODEL PERFORMANCE SUMMARY")
print("="*50)
print(f"   Accuracy: {accuracy_score(y_test, y_pred)*100:.1f}%")
print(f"   ROC-AUC: {roc_auc:.3f}")
print(f"   Optimal Threshold: {optimal_threshold:.3f}")
print(f"\n💡 AUC Interpretation:")
print("   • AUC = 0.5: Random guessing (useless)")
print("   • AUC = 0.7-0.8: Acceptable discrimination")
print("   • AUC = 0.8-0.9: Excellent discrimination")
print("   • AUC > 0.9: Outstanding discrimination")

## ⚖️ Step 5: Threshold Optimization (Medical Cost-Sensitive)

In medicine, **threshold = 0.5 is often WRONG!**

Consider the costs:
- **False Negative (missing disease)**: Patient goes untreated → Potentially FATAL
- **False Positive (false alarm)**: Patient gets extra tests → Inconvenient but safe

> **Medical Decision Rule**: When in doubt, err on the side of caution!

This means we should use a LOWER threshold to maximize **Recall** (catch all sick patients).

In [ ]:
# ═══════════════════════════════════════════════════════════════
# ⚖️ THRESHOLD OPTIMIZATION FOR MEDICAL DIAGNOSIS
# ═══════════════════════════════════════════════════════════════

# Get precision-recall curve
precision_curve, recall_curve, thresholds_pr = precision_recall_curve(y_test, y_proba)

# Calculate F1 and F2 (F2 weights recall more heavily - better for medical)
f1_scores = 2 * (precision_curve[:-1] * recall_curve[:-1]) / (precision_curve[:-1] + recall_curve[:-1] + 1e-10)
f2_scores = 5 * (precision_curve[:-1] * recall_curve[:-1]) / (4 * precision_curve[:-1] + recall_curve[:-1] + 1e-10)

# Find optimal thresholds
optimal_f1_idx = np.argmax(f1_scores)
optimal_f2_idx = np.argmax(f2_scores)

# For medical: we want high recall (catch all cases)
# Find threshold where recall >= 0.9
high_recall_idx = np.where(recall_curve[:-1] >= 0.9)[0]
if len(high_recall_idx) > 0:
    medical_threshold = thresholds_pr[high_recall_idx[-1]]
else:
    medical_threshold = 0.3

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Plot 1: Precision-Recall Curve
ax = axes[0]
ax.plot(recall_curve, precision_curve, 'b-', linewidth=2)
ax.scatter(recall_curve[optimal_f1_idx], precision_curve[optimal_f1_idx], 
          color='red', s=100, label=f'Best F1 (thresh={thresholds_pr[optimal_f1_idx]:.2f})')
ax.set_xlabel('Recall', fontsize=12)
ax.set_ylabel('Precision', fontsize=12)
ax.set_title('Precision-Recall Curve', fontsize=14)
ax.legend()
ax.grid(True, alpha=0.3)

# Plot 2: Metrics vs Threshold
ax = axes[1]
ax.plot(thresholds_pr, f1_scores, 'g-', linewidth=2, label='F1 Score')
ax.plot(thresholds_pr, f2_scores, 'b-', linewidth=2, label='F2 Score (recall-heavy)')
ax.plot(thresholds_pr, recall_curve[:-1], 'r-', linewidth=2, label='Recall')
ax.axvline(x=0.5, color='gray', linestyle='--', alpha=0.5, label='Default (0.5)')
ax.axvline(x=medical_threshold, color='red', linestyle='--', 
          label=f'Medical (90% recall): {medical_threshold:.2f}')
ax.set_xlabel('Threshold', fontsize=12)
ax.set_ylabel('Score', fontsize=12)
ax.set_title('Metrics vs Threshold', fontsize=14)
ax.legend()
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Compare predictions at different thresholds
print("📊 THRESHOLD COMPARISON")
print("="*60)
for thresh, name in [(0.5, 'Default (0.5)'), (medical_threshold, f'Medical ({medical_threshold:.2f})')]:
    y_pred_thresh = (y_proba >= thresh).astype(int)
    tp = np.sum((y_test == 1) & (y_pred_thresh == 1))
    fn = np.sum((y_test == 1) & (y_pred_thresh == 0))
    fp = np.sum((y_test == 0) & (y_pred_thresh == 1))
    recall_val = tp / (tp + fn) if (tp + fn) > 0 else 0
    precision_val = tp / (tp + fp) if (tp + fp) > 0 else 0
    f1_val = 2 * precision_val * recall_val / (precision_val + recall_val) if (precision_val + recall_val) > 0 else 0
    print(f"\n{name}:")
    print(f"   Recall: {recall_val:.3f} (catching {tp}/{tp+fn} sick patients)")
    print(f"   Precision: {precision_val:.3f}")
    print(f"   F1: {f1_val:.3f}")
    print(f"   Missed Cases (FN): {fn}")

print("\n💡 RECOMMENDATION: Use lower threshold in medical screening!")
print("   Better to have extra tests (FP) than miss a disease (FN)!")

## 📈 Step 6: Cross-Validation for Robust Evaluation

A single train-test split can be misleading. Let's use **Stratified K-Fold Cross-Validation**:
- Maintains class balance in each fold
- Gives us confidence intervals for our metrics
- Essential for medical applications where reliability matters

## 🏥 Step 7: Clinical Decision Support System

Let's create a practical tool that could be used in a clinical setting.

This demonstrates how our model would work in production:
1. Take patient data
2. Calculate risk probability
3. Provide actionable recommendations

> **Note**: This is for educational purposes. Real medical AI requires extensive validation and regulatory approval!

In [ ]:
# ═══════════════════════════════════════════════════════════════
# 🏥 CLINICAL DECISION SUPPORT DEMO
# ═══════════════════════════════════════════════════════════════

def predict_heart_disease_risk(patient_data, model, scaler, feature_names):
    """
    Predict heart disease risk for a patient.
    Returns probability and risk category.
    
    Parameters:
    -----------
    patient_data : array-like
        Patient's clinical measurements (unscaled)
    model : trained LogisticRegression model
    scaler : fitted StandardScaler
    feature_names : list of feature names
    
    Returns:
    --------
    tuple: (probability, risk_level, color, recommendation)
    """
    # Convert to DataFrame
    X_patient = pd.DataFrame([patient_data], columns=feature_names)
    
    # Scale
    X_scaled = scaler.transform(X_patient)
    
    # Predict probability
    prob = model.predict_proba(X_scaled)[0, 1]
    
    # Risk stratification (based on clinical guidelines)
    if prob < 0.2:
        risk_level = "LOW RISK"
        color = "🟢"
        recommendation = "Continue healthy lifestyle. Regular annual checkups."
    elif prob < 0.5:
        risk_level = "MODERATE RISK"
        color = "🟡"
        recommendation = "Lifestyle modifications recommended. Monitor closely."
    elif prob < 0.8:
        risk_level = "HIGH RISK"
        color = "🟠"
        recommendation = "Immediate medical consultation. Consider further testing."
    else:
        risk_level = "CRITICAL RISK"
        color = "🔴"
        recommendation = "URGENT: Cardiology referral required immediately."
    
    return prob, risk_level, color, recommendation


# Demo with test samples
print("🏥 CLINICAL DECISION SUPPORT DEMO")
print("="*60)
print("Demonstrating how the model would work in a clinical setting...\n")

# Convert X_test back to DataFrame for easier handling
X_test_df = pd.DataFrame(
    scaler.inverse_transform(X_test) if hasattr(X_test, '__len__') else X_test,
    columns=X.columns
)

# Evaluate on a few test samples
for i in range(min(5, len(X_test))):
    patient = X_test_df.iloc[i].values
    actual = y_test.iloc[i] if hasattr(y_test, 'iloc') else y_test[i]
    
    prob, level, color, rec = predict_heart_disease_risk(
        patient, model, scaler, X.columns.tolist()
    )
    
    actual_status = '❌ Has Heart Disease' if actual == 1 else '✅ No Heart Disease'
    correct = "✓" if (prob >= 0.5 and actual == 1) or (prob < 0.5 and actual == 0) else "✗"
    
    print(f"┌{'─'*56}┐")
    print(f"│ 📋 PATIENT {i+1:3d}                                          │")
    print(f"├{'─'*56}┤")
    print(f"│ {color} Risk Level: {level:20s}                │")
    print(f"│ Probability: {prob:5.1%}                                  │")
    print(f"│ Actual: {actual_status:35s}     │")
    print(f"│ Prediction: {correct}                                       │")
    print(f"│ 📝 {rec[:50]:50s} │")
    print(f"└{'─'*56}┘\n")

In [ ]:
# ═══════════════════════════════════════════════════════════════
# 📈 STRATIFIED K-FOLD CROSS-VALIDATION
# ═══════════════════════════════════════════════════════════════

# Create pipeline (proper way - fit scaler on each fold's training data only)
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('classifier', LogisticRegression(max_iter=1000, random_state=42))
])

# Get full data (unscaled)
X_full = df.drop('target', axis=1)
y_full = (df['target'] == '2').astype(int)

# Cross-validation setup
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Evaluate multiple metrics
metrics = ['accuracy', 'precision', 'recall', 'f1', 'roc_auc']
print("📊 5-FOLD STRATIFIED CROSS-VALIDATION")
print("="*50)

results = {}
for metric in metrics:
    scores = cross_val_score(pipeline, X_full, y_full, cv=cv, scoring=metric)
    results[metric] = scores
    print(f"{metric.upper():12}: {scores.mean():.4f} ± {scores.std():.4f}")

# Visualize results
plt.figure(figsize=(10, 6))
positions = np.arange(len(metrics))
means = [results[m].mean() for m in metrics]
stds = [results[m].std() for m in metrics]

bars = plt.bar(positions, means, yerr=stds, capsize=5, 
               color=['steelblue', 'coral', 'green', 'purple', 'gold'],
               edgecolor='black', linewidth=1.5)
plt.xticks(positions, [m.upper() for m in metrics], fontsize=11)
plt.ylabel('Score', fontsize=12)
plt.title('Cross-Validation Results (5-Fold Stratified)\nError bars show ±1 std dev', fontsize=14)
plt.ylim(0, 1.1)

# Add value labels on bars
for bar, mean, std in zip(bars, means, stds):
    plt.text(bar.get_x() + bar.get_width()/2, bar.get_height() + std + 0.02,
             f'{mean:.3f}', ha='center', fontsize=10, fontweight='bold')

plt.tight_layout()
plt.show()

print("\n💡 These confidence intervals help us understand model reliability!")
print("   Low variance = consistent performance across folds")
print("   High variance = model may be sensitive to data splits")

In [ ]:
# ═══════════════════════════════════════════════════════════════
# ⚖️ THRESHOLD OPTIMIZATION FOR MEDICAL DIAGNOSIS
# ═══════════════════════════════════════════════════════════════

# Get precision-recall curve
precision, recall, thresholds_pr = precision_recall_curve(y_test, y_proba)

# Calculate F1 and F2 (F2 weights recall more heavily)
f1_scores = 2 * (precision[:-1] * recall[:-1]) / (precision[:-1] + recall[:-1] + 1e-10)
f2_scores = 5 * (precision[:-1] * recall[:-1]) / (4 * precision[:-1] + recall[:-1] + 1e-10)

# Find optimal thresholds
optimal_f1_idx = np.argmax(f1_scores)
optimal_f2_idx = np.argmax(f2_scores)

# For medical: we want high recall (catch all cases)
# Find threshold where recall >= 0.9
high_recall_idx = np.where(recall[:-1] >= 0.9)[0]
if len(high_recall_idx) > 0:
    medical_threshold = thresholds_pr[high_recall_idx[-1]]
else:
    medical_threshold = 0.3

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Plot 1: Precision-Recall Curve
ax = axes[0]
ax.plot(recall, precision, 'b-', linewidth=2)
ax.scatter(recall[optimal_f1_idx], precision[optimal_f1_idx], 
          color='red', s=100, label=f'Best F1 (thresh={thresholds_pr[optimal_f1_idx]:.2f})')
ax.set_xlabel('Recall', fontsize=12)
ax.set_ylabel('Precision', fontsize=12)
ax.set_title('Precision-Recall Curve', fontsize=14)
ax.legend()
ax.grid(True, alpha=0.3)

# Plot 2: Metrics vs Threshold
ax = axes[1]
ax.plot(thresholds_pr, f1_scores, 'g-', linewidth=2, label='F1 Score')
ax.plot(thresholds_pr, f2_scores, 'b-', linewidth=2, label='F2 Score (recall-heavy)')
ax.plot(thresholds_pr, recall[:-1], 'r-', linewidth=2, label='Recall')
ax.axvline(x=0.5, color='gray', linestyle='--', alpha=0.5, label='Default (0.5)')
ax.axvline(x=medical_threshold, color='red', linestyle='--', 
          label=f'Medical (90% recall): {medical_threshold:.2f}')
ax.set_xlabel('Threshold', fontsize=12)
ax.set_ylabel('Score', fontsize=12)
ax.set_title('Metrics vs Threshold', fontsize=14)
ax.legend()
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Compare predictions at different thresholds
print("📊 THRESHOLD COMPARISON")
print("="*60)
for thresh, name in [(0.5, 'Default (0.5)'), (medical_threshold, f'Medical ({medical_threshold:.2f})')]:
    y_pred_t = (y_proba >= thresh).astype(int)
    tp = np.sum((y_test == 1) & (y_pred_t == 1))
    fn = np.sum((y_test == 1) & (y_pred_t == 0))
    fp = np.sum((y_test == 0) & (y_pred_t == 1))
    recall_t = tp / (tp + fn) if (tp + fn) > 0 else 0
    precision_t = tp / (tp + fp) if (tp + fp) > 0 else 0
    f1_t = 2 * precision_t * recall_t / (precision_t + recall_t) if (precision_t + recall_t) > 0 else 0
    print(f"\n{name}:")
    print(f"   Recall: {recall_t:.3f} (catching {tp}/{tp+fn} sick patients)")
    print(f"   Precision: {precision_t:.3f}")
    print(f"   F1: {f1_t:.3f}")
    print(f"   Missed Cases (FN): {fn}")

print("\n💡 RECOMMENDATION: Use lower threshold in medical screening!")
print("   Better to have extra tests (FP) than miss a disease (FN)!")

## 🎓 Conclusion & Key Takeaways

### What We Learned

1. **Logistic Regression is ideal for medical diagnosis** because it provides interpretable coefficients
2. **Probability outputs** help doctors make nuanced decisions (not just yes/no)
3. **ROC-AUC is better than accuracy** for imbalanced medical data
4. **Threshold tuning is CRITICAL** - in medicine, we prioritize recall to catch all cases
5. **Cross-validation** gives us confidence in model reliability

### Medical ML Best Practices

| Aspect | Recommendation |
|--------|---------------|
| **Threshold** | Lower than 0.5 for screening (maximize recall) |
| **Metrics** | Focus on Recall, F2-score, ROC-AUC |
| **Validation** | Stratified K-Fold CV with multiple metrics |
| **Interpretation** | Use coefficients to expother risk factors |
| **Deployment** | Always include uncertainty estimates |

### Ethical Considerations

> *"The goal of ML in healthcare is to AUGMENT human decision-making, not replace it."*

- ✅ Help doctors prioritize high-risk patients
- ✅ Provide second opinion for diagnosis
- ❌ Never make final medical decisions autonomously
- ❌ Never deploy without proper clinical validation

### Next Steps

1. Validate on independent dataset
2. Test across different patient populations
3. Get clinical expert review
4. Submit for regulatory approval (FDA, CE marking)

---

**Congratulations!** You've built a production-ready heart disease classifier! 🎉

## 🚀 Production Checklist

Before deploying any medical ML model:

- [ ] **Data Quality**: Verify no data leakage, proper train/test split
- [ ] **Clinical Validation**: Reviewed by domain experts
- [ ] **Bias Testing**: Tested across demographics (age, sex, ethnicity)
- [ ] **Threshold Calibration**: Optimized for clinical use case
- [ ] **Expotherability**: Can expother predictions to clinicians
- [ ] **Monitoring**: Track performance drift over time
- [ ] **Regulatory**: Compliance with medical device regulations (FDA, CE)
- [ ] **Documentation**: Complete model card and usage guidelines

---

### 📚 Further Reading

- **Scikit-learn**: [Logistic Regression Documentation](https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression)
- **Medical ML**: "Machine Learning in Healthcare" by MIT
- **Interpretability**: "Interpretable Machine Learning" by Christoph Molnar
- **Regulatory**: FDA guidance on AI/ML-based Software as a Medical Device (SaMD)

---

*Built with ❤️ for education | Not for clinical use without proper validation*